In [1]:
import specpy as sp

In [3]:
i = sp.get_application()
print(i.version())

# create new measurement
i.create_measurement()
ms = i.active_measurement()

16.3.11958-w2026


In [8]:
# starts measurement, but errors afterwards, does not block
i.run(ms)

0

In [7]:
# hacky fix

from threading import Semaphore

def blocking_imspector_run(imspector, measurement):
    '''
    since imspector.run(measurement) seems buggy a.t.m. and does not block,
    this wrapps it into a blocking call.
    
    also catches the 'bad cast' exception that seems to always happen
    '''
        
    sem = Semaphore()
    sem.acquire()
    imspector.connect_end(sem.release, 0)

    try:
        imspector.run(measurement)
    except RuntimeError as e:
        # bad cast error is always thrown, but measurement seems to work nonetheless
        # in that specific case, we ignore the error
        if str(e) == 'bad cast':
            pass
        else:
            raise

    sem.acquire()
    imspector.disconnect_end(sem.release, 0)

blocking_imspector_run(i, ms)